# 実験及び演習 第1ターム 関先生

マーケットバスケット分析 + ナップサックDP

## 開発したシステムは以下

- 入力して整形
- マーケットバスケット分析
- ナップサックDP


In [ ]:
""" check hardware info """ 
!lscpu
!nvidia-smi


In [ ]:
""" install library """
!pip install numpy pandas matplotlib mlxtend tqdm


""" import library """
from tqdm.notebook import tqdm # プログレスバーを表示する
from logging import Logger
from logging import getLogger
from logging import basicConfig
from logging import DEBUG
from logging import StreamHandler
import os
import datetime
from logging import FileHandler
import pandas as pd
from collections import defaultdict
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [ ]:
""" init logger """
logger = getLogger("logger")
logger.setLevel(DEBUG)
basicConfig(
    level=DEBUG,
    datefmt='%H:%M:%S-%Y/%m/%d',
    format='[logger/%(levelname)s] %(asctime)s | %(funcName)s/l:%(lineno)s | msg:%(message)s',
)


In [ ]:
""" load data """
# 実行時間：1年分で約30秒
logger.info("start:read csv")
# csv_dataset: pd.DataFrame = pd.read_csv("/content/drive/MyDrive/実験及び演習/data-month.csv", encoding="UTF-8", dtype=str) # 1ヶ月分
csv_dataset: pd.DataFrame = pd.read_csv("/content/drive/MyDrive/実験及び演習/data-year.csv", encoding="UTF-8", dtype=str) # 1年分
display(csv_dataset.info())
# display(csv_dataset.head(3))
logger.info("end:read csv")


In [ ]:
""" create dictionary """
# 実行時間：1年分で約10分(563万件/O(10**6))
logger.info("start:create dict")
dataset = defaultdict(list)
classified_names = set()
prices = defaultdict(int)
for i in tqdm(range(len(csv_dataset))):
    receipt = csv_dataset.loc[i]
    if str(receipt["購入商品名"]).strip() == "nan" or str(receipt["分類名"]).strip() == "新規アイテム" or str(receipt["単価"]) == "単価":
        # ごみデータをスキップ
        continue
    classified_name = str(receipt["分類名"]).strip().replace("\u3000", "").replace(" ", "")
    classified_names.add(classified_name)
    if prices[classified_name]: # 既に登録していた場合は平均を取る
        prices[classified_name] = ((prices[classified_name] + int(receipt["単価"])) // 2)
    else:
        prices[classified_name] = int(receipt["単価"])   
    dataset[receipt["レシート番号"]].append(classified_name)
dataset = dataset.values()
# print(sorted(prices.items(), key=lambda k: k[1], reverse=True)[:-100]) # 単価=0が無いことを確認


""" create dataframe """
logger.info("start:create df")
# ref: http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_array, columns=te.columns_)


""" save checkpoint """
with open("/content/drive/MyDrive/実験及び演習/classified_names.pickle", "wb") as f:
    pickle.dump(classified_names, f)
with open("/content/drive/MyDrive/実験及び演習/prices.pickle", "wb") as f:
    pickle.dump(prices, f)
df.to_pickle("/content/drive/MyDrive/実験及び演習/dataframe_data_year.pickle") # dataframeを保存


In [ ]:
""" check """
display(df.info())
display(df.head())
df.shape
display(list(prices.items())[:10])
print("classified names size:", len(classified_names))


In [ ]:
""" apriori """
# ref: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
# 実行時間：1年分で約30分(min_support=0.001の場合)
# 実行時間：1年分で約4分(min_support=0.005の場合)
logger.info("start:apriori")
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
# frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True) # 1年分で143個
# frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True) # 1年分で59個
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # "length" 列を追加
frequent_itemsets = frequent_itemsets[(frequent_itemsets["length"] <= 2)] # lengthが2以下のものを抽出
frequent_itemsets.to_pickle("/content/drive/MyDrive/実験及び演習/frequent_itemsets_year_0001.pickle") # dataframeを保存
logger.info("end:apriori")
# print("frequent itemsets")
# display(frequent_itemsets)


In [ ]:
""" check """
from collections import Counter
print(Counter(frequent_itemsets["length"]))

# pd.set_option('display.max_rows', None)
frequent_itemsets.loc[:, ["itemsets", "length"]]
pd.set_option('display.max_rows', 10)


In [ ]:
""" association rules """
# ref: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
# 実行時間：1年分でも1秒
logger.info("start:association")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)
rules = rules[(rules["lift"] >= 1)] # lift値が1未満のものは不適切なので除去
logger.info("end:association")
# print("rules")
# display(rules)
display(rules.sort_values("lift", ascending=False).loc[:, ["antecedents", "consequents", "lift"]])


In [ ]:
def recommendation_system() -> bool:
    """ input user select """
    while True:
        user_select = input("user select (enter \"exit\" to exit):").strip()
        if user_select == "exit":
            print("ok. bye")
            return True
        if user_select in classified_names:
            break
        else:
            print("Please select from the products you have purchased in the past")
            print("select agein ? (y/n)")
            if input() == "y":
                continue
            user_select = classified_names.pop()
            classified_names.add(user_select) # 参照したいだけなので，元に戻す
            break
    print("user_select:", user_select)


    """ create values list """
    values = defaultdict(int)
    for antecedents, consequents, confidence in zip(rules["antecedents"], rules["consequents"], rules["confidence"]):
        antecedents = "".join(list(antecedents)).strip()
        consequents = "".join(list(consequents)).strip()
        if user_select in antecedents:
            values[consequents] = int(confidence * 1000)
    classified_num = len(values)
    print("candidate size:", classified_num)
    if classified_num == 0:
        return False


    """ knapsack dp """
    # logger.info("start:knapsack dp")
    max_cost = min(10**4, int(input("input max cost (between 10 and 10000):")))
    print(f"size of dp: {(classified_num + 1)} * {(max_cost + 1)} = {(classified_num + 1) * (max_cost + 1)}")
    # dpとprevの大きさは10^4 * 10^4 くらいならok (空間計算量，時間計算量がO(NM)なので)

    dp = [[0 for _ in range(max_cost + 1)] for _ in range(classified_num + 1)]
    # dp[i][j] := i番目までの商品で，予算j以内とした時の最大の利益
    prev = [[0 for _ in range(max_cost + 1)] for _ in range(classified_num + 1)]
    # prev[i][j] := dp[i + 1][j]がdp[i][prev[i + 1][j]]で更新された

    for i, (classified_name, value) in enumerate(values.items()):
        price = prices[classified_name]
        for cost in range(max_cost + 1):
            if price <= cost:
                if dp[i + 1][cost] < dp[i][cost - price] + value:
                    dp[i + 1][cost] = dp[i][cost - price] + value
                    prev[i + 1][cost] = cost - price
            if dp[i + 1][cost] < dp[i][cost]:
                dp[i + 1][cost] = dp[i][cost]
                prev[i + 1][cost] = cost

    """ recovery of optimal solution """
    value_keys = list(values.keys())
    current_cost = max_cost
    opt_set = set()
    used_cost = 0
    for i in reversed(range(classified_num)):
        if prev[i + 1][current_cost] == current_cost - prices[value_keys[i]]:
            opt_set.add(value_keys[i])
            used_cost += int(prices[value_keys[i]])
        current_cost = prev[i + 1][current_cost]
    # logger.info("end:knapsack dp")


    """ printout result """
    print("-------------------------------------------------------------------")
    print("user select:", user_select)
    print("candidate size:", classified_num)
    print("candidate:", values)
    print("recommend:")
    print(f"\tused cost:{used_cost} / {max_cost}")
    print("\tproduct:")
    minus = 0
    for opt in opt_set:
        if prices[opt] == 0:
            minus += prices[opt]
            # continue
        print(f"\t\tcost(price):{prices[opt]} / value(confidence):{values[opt]} / classified name:{opt}")
    print("value(dp[classified_num][max_cost]):", dp[classified_num][max_cost] - minus)
    print("-------------------------------------------------------------------\n\n\n")
    return False


if __name__ == "__main__":
    while True:
        if recommendation_system():
            break



## 成果発表会のデモンストレーション用
前処理からマーケットバスケット分析までは事前に作成したpickleデータを使用する．
DPと推薦システムの動作を見せる

In [ ]:
""" check hardware info """ 
!lscpu
!nvidia-smi

""" install library """
!pip install numpy pandas matplotlib mlxtend tqdm

""" import library """
from tqdm.notebook import tqdm # プログレスバーを表示する
from logging import Logger
from logging import getLogger
from logging import basicConfig
from logging import DEBUG
from logging import StreamHandler
import os
import datetime
from logging import FileHandler
import pandas as pd
from collections import defaultdict
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


In [ ]:
""" init logger """
logger = getLogger("logger")
logger.setLevel(DEBUG)
basicConfig(
    level=DEBUG,
    datefmt='%H:%M:%S-%Y/%m/%d',
    format='[logger/%(levelname)s] %(asctime)s | %(funcName)s/l:%(lineno)s | msg:%(message)s',
)


""" load pickle data """
logger.info("start:load pickle data")
with open("/content/drive/MyDrive/実験及び演習/classified_names.pickle", "rb") as f:
    classified_names = pickle.load(f)
with open("/content/drive/MyDrive/実験及び演習/prices.pickle", "rb") as f:
    prices = pickle.load(f)
frequent_itemsets = pd.read_pickle("/content/drive/MyDrive/実験及び演習/frequent_itemsets_year_0001.pickle")
logger.info("end:load pickle data")


""" association rules """
logger.info("start:association")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)
rules = rules[(rules["lift"] >= 1)] # lift値が1未満のものは不適切なので除去
logger.info("end:association")


def recommendation_system() -> bool:
    """ input user select """
    while True:
        user_select = input("user select (enter \"exit\" to exit):").strip()
        if user_select == "exit":
            print("ok. bye")
            return True
        if user_select in classified_names:
            break
        else:
            print("Please select from the products you have purchased in the past")
            print("select agein ? (y/n)")
            if input() == "y":
                continue
            user_select = classified_names.pop()
            classified_names.add(user_select) # 参照したいだけなので，元に戻す
            break
    print("user_select:", user_select)


    """ create values list """
    values = defaultdict(int)
    for antecedents, consequents, confidence in zip(rules["antecedents"], rules["consequents"], rules["confidence"]):
        antecedents = "".join(list(antecedents)).strip()
        consequents = "".join(list(consequents)).strip()
        if user_select in antecedents:
            values[consequents] = int(confidence * 1000)
    classified_num = len(values)
    print("candidate size:", classified_num)
    if classified_num == 0:
        return False


    """ knapsack dp """
    # logger.info("start:knapsack dp")
    max_cost = min(10**4, int(input("input max cost (between 10 and 10000):")))
    print(f"size of dp: {(classified_num + 1)} * {(max_cost + 1)} = {(classified_num + 1) * (max_cost + 1)}")
    # dpとprevの大きさは10^4 * 10^4 くらいならok (空間計算量，時間計算量がO(NM)なので)

    dp = [[0 for _ in range(max_cost + 1)] for _ in range(classified_num + 1)]
    # dp[i][j] := i番目までの商品で，予算j以内とした時の最大の利益
    prev = [[0 for _ in range(max_cost + 1)] for _ in range(classified_num + 1)]
    # prev[i][j] := dp[i + 1][j]がdp[i][prev[i + 1][j]]で更新された

    for i, (classified_name, value) in enumerate(values.items()):
        price = prices[classified_name]
        for cost in range(max_cost + 1):
            if price <= cost:
                if dp[i + 1][cost] < dp[i][cost - price] + value:
                    dp[i + 1][cost] = dp[i][cost - price] + value
                    prev[i + 1][cost] = cost - price
            if dp[i + 1][cost] < dp[i][cost]:
                dp[i + 1][cost] = dp[i][cost]
                prev[i + 1][cost] = cost

    """ recovery of optimal solution """
    value_keys = list(values.keys())
    current_cost = max_cost
    opt_set = set()
    used_cost = 0
    for i in reversed(range(classified_num)):
        if prev[i + 1][current_cost] == current_cost - prices[value_keys[i]]:
            opt_set.add(value_keys[i])
            used_cost += int(prices[value_keys[i]])
        current_cost = prev[i + 1][current_cost]
    # logger.info("end:knapsack dp")


    """ printout result """
    print("-------------------------------------------------------------------")
    print("user select:", user_select)
    print("candidate size:", classified_num)
    print("candidate:", values)
    print("recommend:")
    print(f"\tused cost:{used_cost} / {max_cost}")
    print("\tproduct:")
    minus = 0
    for opt in opt_set:
        if prices[opt] == 0:
            minus += prices[opt]
            # continue
        print(f"\t\tcost(price):{prices[opt]} / value(confidence):{values[opt]} / classified name:{opt}")
    print("value(dp[classified_num][max_cost]):", dp[classified_num][max_cost] - minus)
    print("-------------------------------------------------------------------\n\n\n")
    return False


if __name__ == "__main__":
    while True:
        if recommendation_system():
            break
